In [1]:
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import pandas as pd
import seq_generator
import tensorflow.keras.layers as layers
import tensorflow_datasets as tfds

2022-08-23 18:06:33.011590: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 18:06:33.011617: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/felix/anaconda3/envs/r/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_ds , val_ds = seq_generator.get_train_and_val(batch_size=32)

2022-08-23 18:06:37.546059: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-23 18:06:37.546089: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-23 18:06:37.546119: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (felix-NBLK-WAX9X): /proc/driver/nvidia/version does not exist
2022-08-23 18:06:37.546536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
cnn = tf.keras.models.load_model("cnn_base/model")

In [5]:
def build_model(hp):


    inputs = tf.keras.Input([5,260,260,3])
    x = layers.TimeDistributed(cnn)(inputs)

    drop1 = hp.Float("drop1",min_value = 0,max_value = 0.4)

    x = layers.TimeDistributed( layers.Dropout(drop1), name = "drop1") (x)

    hidden_units =  hp.Int("hidden_units",min_value=64, max_value=1028, step=64)

    if hp.Choice("rnn",["lstm","gru"]) == "lstm":
        x = layers.LSTM(hidden_units, name = "lstm")(x)
    else:
        x = layers.GRU(hidden_units, name = "gru") (x)

    drop2 = hp.Float("drop2",min_value = 0,max_value = 0.4)
    x = layers.Dropout(drop2,name ="drop2")(x)

    outputs = tf.keras.layers.Dense(2, activation="softmax", name="pred",kernel_regularizer=tf.keras.regularizers.L2())(x)

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model = tf.keras.Model(inputs,outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

build_model(kt.HyperParameters())

In [6]:
!mkdir tuner_run

In [7]:
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory='tuner_run',
                     project_name='vd_seq')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_ds, epochs=200, validation_data=val_ds, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")